# ODTC Tutorial: Connect, Lid, Block Temperature, and Protocol

This notebook walks through the ODTC (Inheco) thermocycler interface: setup, listing methods, lid (door) commands, setting block temperature, and running a protocol. Use it as a reference for the recommended workflow.

## 1. Imports and thermocycler

Use **ODTCThermocycler** (recommended): it owns `odtc_ip`, `variant`, and dimensions. Alternative: generic `Thermocycler` + `ODTCBackend` for custom backend options.

In [ ]:
import logging
from pylabrobot.resources import Coordinate
from pylabrobot.thermocycling.inheco import ODTCThermocycler
# Preferred: ODTCThermocycler (dimensions from ODTC_DIMENSIONS; variant 96 or 384)
tc = ODTCThermocycler(name="odtc_test", odtc_ip="192.168.1.50", variant=96, child_location=Coordinate.zero())
# Override: tc = Thermocycler(..., backend=ODTCBackend(odtc_ip=..., variant=96, logger=...), ...) for custom backend

## 2. Connect and list device methods

`setup()` resets and initializes the device. **Methods** = runnable protocols on the device; **premethods** = setup-only (e.g. set temperature). Use `list_protocols()` for names; `get_protocol(name)` returns a `StoredProtocol` for methods (None for premethods).

In [ ]:
await tc.setup()
print("✓ Connected and initialized.")

**After `get_protocol(name)`** you get a `StoredProtocol` (`.protocol` + `.config` with overshoot, PID, etc.):

- **Roundtrip:** `run_protocol(stored.protocol, block_max_volume, config=stored.config)` — same device-calculated config.
- **Run by name (recommended for PCR):** `run_stored_protocol("MethodName")` — device runs its Script Editor method; optimal thermal (overshoots utilized, device-tuned ramps).
- **Weaker option:** Uploading a custom protocol via `run_protocol(protocol, block_max_volume)` **without** a corresponding calculated config — no device overshoot/PID, so thermal performance is not optimized.

In [ ]:
protocol_names = await tc.backend.list_protocols()
print(f"Device methods/premethods ({len(protocol_names)}):", protocol_names[:10], "..." if len(protocol_names) > 10 else "")

# Optional: inspect a runnable method (get_protocol returns None for premethods)
# StoredProtocol has .protocol (Protocol) and .config (ODTCConfig: overshoot, PID, etc.)
if protocol_names:
    stored = await tc.backend.get_protocol(protocol_names[0])
    if stored:
        print(f"Example: {stored.name} (variant {stored.config.variant}) — {len(stored.protocol.stages)} stage(s)")
        # Roundtrip: run with same ODTC config (overshoot, PID) via run_protocol(stored.protocol, block_max_volume, config=stored.config)


## 3. Lid (door) commands

The Thermocycler API uses **`open_lid`** / **`close_lid`** (ODTC device calls this the door). Use **`wait=False`** to get an execution handle and avoid blocking; then **`await handle.wait()`** or **`await handle`** when you need to wait. Omit `wait=False` to block until the command finishes.

**Waiting:** `await handle.wait()` or `await handle` (same). For method/protocol runs, **`await tc.wait_for_profile_completion(poll_interval=..., timeout=...)`** uses polling and supports a timeout.

In [ ]:
# Non-blocking: returns CommandExecution handle; await handle.wait() when you need to wait
door_handle = await tc.close_lid(wait=False)
print(f"Close started (request_id={door_handle.request_id})")

## 4. Block temperature and protocol

ODTC has no direct “set block temp” command; **`set_block_temperature`** uploads and runs a PreMethod. Use **`wait=False`** to get a handle; **`run_protocol(protocol, block_max_volume, config=...)`** is always non-blocking — await the returned handle or **`tc.wait_for_profile_completion()`** to block. Use **`config=tc.backend.get_default_config(post_heating=True)`** to hold temperatures after the method ends.

In [ ]:
await door_handle.wait()
print("Door closed.")

# set_block_temperature runs a premethod; wait=False returns MethodExecution handle
# Override: debug_xml=True, xml_output_path="out.xml" to save generated MethodSet XML
mount_handle = await tc.set_block_temperature([37.0],
                                              wait=False,
                                              debug_xml=True,
                                              xml_output_path="debug_set_mount_temp.xml"
                                              )
block = await tc.get_block_current_temperature()
lid = await tc.get_lid_current_temperature()
print(f"Block: {block[0]:.1f} °C  Lid: {lid[0]:.1f} °C")

In [ ]:
# Poll temps while method runs (optional)
block, lid = await tc.get_block_current_temperature(), await tc.get_lid_current_temperature()
print(f"Block: {block[0]:.1f} °C  Lid: {lid[0]:.1f} °C")

In [ ]:
from pylabrobot.thermocycling.standard import Protocol, Stage, Step

# Wait for set_block_temperature (previous cell) to finish before starting a protocol
await mount_handle

# run_protocol is always non-blocking; returns MethodExecution. To block: await handle.wait() or tc.wait_for_profile_completion()
config = tc.backend.get_default_config(post_heating=False)  # if True: hold temps after method ends
cycle_protocol = Protocol(stages=[
    Stage(steps=[
        Step(temperature=[37.0], hold_seconds=10.0),
        Step(temperature=[60.0], hold_seconds=10.0),
        Step(temperature=[10.0], hold_seconds=10.0),
    ], repeats=1)
])
execution = await tc.run_protocol(cycle_protocol, 50.0, config=config)
# Override: run_stored_protocol("MethodName") to run a device-stored method by name
print(f"Protocol started (request_id={execution.request_id})")
block, lid = await tc.get_block_current_temperature(), await tc.get_lid_current_temperature()
print(f"Block: {block[0]:.1f} °C  Lid: {lid[0]:.1f} °C")


## 5. Wait, open lid, disconnect

Await protocol completion, open the lid (non-blocking then wait), then **`tc.stop()`** to close the connection.

In [ ]:
# Block until protocol done (alternatively: await tc.wait_for_profile_completion(poll_interval=..., timeout=...))
await execution.wait()

open_handle = await tc.open_lid(wait=False)
await open_handle.wait()
await tc.stop()
print("Done.")